## Installation et import des librairies

In [ ]:
!pip install twarc
!pip install wget

# Install LDA Libraries
!pip install textblob
!pip install tweepy
!pip install pycountry
!pip install  wordcloud 
!pip install langdetect 
!pip install pyLDAvis

#temp

#!pip install networkx==1.11

In [ ]:
import sys
import os
import requests
import re
import tweepy
from geopy.geocoders import Nominatim
import json
geolocator = Nominatim(user_agent="https://colab.research.google.com/drive/15lPys0g207RL6O9F3n1_U2yhQzBToznj?usp=sharing")

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import plotly.express as px


from textblob import TextBlob
import pycountry
import re
import string
import folium

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from langdetect import detect

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

import gensim
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle

import pandas as pd
import requests
import gzip
import shutil
import wget
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets
import numpy as np
from datetime import datetime  
from datetime import timedelta  

# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
!twarc configure --consumer_key="sCEQKMT4PE6MC3NiorTZpPBMM"y --consumer_secret="s90Ew2cJb32oFyF4EIMdxfotjBQckbm6OkE6hKpMpbKKwCRlAR" --access_token="1358769056766164996-qtIxIlGMiLeOFi2K6MuKxh5MgnI3sc" --access_token_secret="cSSZsEzzbBAAQxGmprgCAWR73OC2U4lJl5Cr9vlbMe9mH"

## Extraction des données

In [ ]:
tweetsDf = pd.DataFrame()
  
#Add 1 day  
date = datetime.strptime("2020-03-20", "%Y-%m-%d") + timedelta(days=1)

for i in range(374):
  date = date + timedelta(days=1)

  #Downloads the dataset (compressed in a GZ format)
  #!wget dataset_URL -O clean-dataset.tsv.gz
  dateStr = str(date.year) + ('-0' + str(date.month) if date.month < 10 else '-' + str(date.month)) + ('-0' + str(date.day) if date.day < 10 else '-' + str(date.day))

  
  url = "https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/" + dateStr + "/" + dateStr + "_clean-dataset.tsv.gz?raw=true"
  print(url)
  wget.download(url, out='clean-dataset.tsv.gz')

  #Unzips the dataset and gets the TSV dataset
  with gzip.open('clean-dataset.tsv.gz', 'rb') as f_in:
      with open('clean-dataset.tsv', 'wb') as f_out:
          shutil.copyfileobj(f_in, f_out)

  #Deletes the compressed GZ file
  os.unlink("clean-dataset.tsv.gz")

  #Gets all possible languages from the dataset
  df = pd.read_csv('clean-dataset.tsv',sep="\t")
  #df = df.sort_values(by='favorite_count', ascending=False)
  tweetsDf = pd.concat([tweetsDf, df.sample(2000)])


In [ ]:
print(tweetsDf.head())

In [ ]:
!pip install pandas==1.1.5

In [ ]:
tweetsDf[["tweet_id"]].to_csv("tweets.csv", index=False, header=False)

In [ ]:
pd.read_csv("tweets.csv").shape

(747999, 1)

In [ ]:
! twarc hydrate tweets.csv > "/content/drive/My Drive/Data/covid-tweets.jsonl"

In [ ]:
tweets = pd.read_json("/content/drive/My Drive/Data/covid-tweets.jsonl", lines=True, chunksize=20000)

In [ ]:
f = open("/content/drive/My Drive/Data/covid-tweets-en.jsonl", 'w')
f.close()

In [ ]:
for chunk in tweets:
  out = chunk[chunk.lang=="en"].to_json(orient='records', lines=True)
  with open("/content/drive/My Drive/Data/covid-tweets-en.jsonl", 'a') as f:
    f.write(out)
    f.write('\n')

In [ ]:
tweetsDf = pd.read_json("/content/drive/My Drive/Data/covid-tweets-en.jsonl", orient='records', lines=True, chunksize=20000)

In [ ]:
lines=0
df = pd.DataFrame()
for chunk in tweetsDf:
  chunk.drop(["truncated", "display_text_range", "in_reply_to_status_id", "in_reply_to_status_id_str",
              "in_reply_to_user_id", "in_reply_to_user_id_str", "in_reply_to_screen_name", "is_quote_status", "quoted_status_id",
              'quoted_status_id_str', 'quoted_status_permalink', 'quoted_status', 'possibly_sensitive', 'extended_entities', 'withheld_in_countries'],axis=1, inplace=True)
  df = pd.concat([df, chunk], axis=0)


In [ ]:
df.shape

(348329, 18)

In [ ]:
df.head()

In [ ]:
df.to_json("/content/drive/My Drive/Data/covid-tweets-final.jsonl", orient="records", lines=True)

## Lecture des tweets à partir du fichier

In [ ]:
tweetsDf = pd.read_json("/content/drive/My Drive/Data/covid-tweets-final.jsonl", orient='records', lines=True, chunksize=20000)

In [ ]:
df = pd.DataFrame()
for chunk in tweetsDf:
  df = pd.concat([df, chunk], axis=0)

In [ ]:
tweetsDf = df.copy()

## Preprocessing

In [ ]:
def remove_urls(text):
  result = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''',' ',text)
  return result

stop_words = stopwords.words('english')
#add punctuation char's to stopwords list
stop_words += list(string.punctuation) # <-- contains !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
#add integers
stop_words += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

def tokenize_lowercase(text):
    tokens = nltk.word_tokenize(text)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stop_words]
    return stopwords_removed
  

def remove_nums(text_object):
    no_nums = list(filter(lambda x: x.isalpha(), text_object))
    return no_nums


lemmatizer = WordNetLemmatizer()
def lemmatize_text(df_text):
    lemmatized =[]
    for w in df_text:
        if(len(w)>3):
          lemmatized.append(lemmatizer.lemmatize(w))

    return lemmatized


def list_to_sentence(list_words):
  return " ".join(list_words)


def preprocess(text):
  return lemmatize_text(remove_nums(tokenize_lowercase(remove_urls(text))))

In [ ]:
tweetsDf['full_text'] = tweetsDf['full_text'].map(remove_urls)
tweetsDf['full_text'] = tweetsDf['full_text'].apply(tokenize_lowercase)
tweetsDf['full_text'] = tweetsDf['full_text'].apply(remove_nums)
tweetsDf['full_text'] = tweetsDf['full_text'].apply(lemmatize_text)

In [ ]:
#tweetsDf.style.set_properties(subset=['full_text'], **{'width': '1000px'})
tweetsDf['full_text'].head()

0    [coronavirus, curfew, get, good, response, pun...
1    [frontliners, must, tested, covid, priority, p...
2    [united, state, department, health, human, ser...
3       [yellowknife, community, foundation, response]
4    [wealthy, calling, doctor, ventilator, manufac...
Name: full_text, dtype: object

## LDA

### Dictionaire de données

In [ ]:
dictionary = gensim.corpora.Dictionary(tweetsDf['full_text'])

In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 coronavirus
1 curfew
2 get
3 good
4 haryana
5 picture
6 punjab
7 response
8 story
9 tribune
10 covid


In [ ]:
dictionary.filter_extremes(no_above=0.8, no_below=15, keep_n=100000)

### Bag of words

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in tweetsDf['full_text']]

### TF-IDF

In [ ]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

for doc in corpus_tfidf[1:]:
    pprint(doc)
    break

[(10, 0.22995012331945805),
 (11, 0.5703723179516612),
 (12, 0.25413644758462967),
 (13, 0.304232896059552),
 (14, 0.1892233219970177),
 (15, 0.41075506795152283),
 (16, 0.41075506795152283),
 (17, 0.302341617017556)]


### Entrainnement du modèle

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Words: {}'.format(idx, topic))

Topic: 0 Words: 0.022*"coronavirus" + 0.020*"year" + 0.009*"pandemic" + 0.008*"china" + 0.008*"covid" + 0.007*"watch" + 0.006*"game" + 0.006*"video" + 0.006*"news" + 0.006*"chinese"
Topic: 1 Words: 0.034*"coronavirus" + 0.030*"trump" + 0.020*"biden" + 0.019*"variant" + 0.016*"vaccine" + 0.015*"american" + 0.015*"president" + 0.014*"dos" + 0.012*"shot" + 0.011*"relief"
Topic: 2 Words: 0.044*"death" + 0.038*"test" + 0.035*"case" + 0.029*"positive" + 0.028*"coronavirus" + 0.022*"covid" + 0.019*"testing" + 0.017*"total" + 0.016*"number" + 0.013*"hospital"
Topic: 3 Words: 0.157*"vaccine" + 0.024*"vaccination" + 0.013*"patient" + 0.012*"first" + 0.012*"worker" + 0.011*"study" + 0.010*"astrazeneca" + 0.009*"coronavirus" + 0.009*"health" + 0.009*"covid"
Topic: 4 Words: 0.020*"people" + 0.015*"country" + 0.014*"government" + 0.009*"world" + 0.009*"coronavirus" + 0.009*"lockdown" + 0.007*"pandemic" + 0.007*"restriction" + 0.007*"travel" + 0.006*"well"
Topic: 5 Words: 0.022*"pandemic" + 0.011*"bu

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

### Prediction du topic d'un nouveau tweet

In [ ]:
unseen_document = 'I’d suggest that neighboring States control their frontiers & ask all Texans who want to cross State lines, have a Covid test! Otherwise all the efforts of good people in neighboring states, wearing masks & being careful, can be thrown in the air by these irresponsible governors!'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("{} - Score: {}\t Topic: {}".format(index, score, lda_model_tfidf.print_topic(index, 5)))

1 - Score: 0.6430994272232056	 Topic: 0.008*"mask" + 0.007*"people" + 0.005*"vaccinated" + 0.005*"like" + 0.004*"know"
3 - Score: 0.13851813971996307	 Topic: 0.026*"case" + 0.018*"death" + 0.012*"coronavirus" + 0.008*"update" + 0.008*"vaccine"
2 - Score: 0.1333116739988327	 Topic: 0.010*"vaccine" + 0.008*"vaccination" + 0.006*"trump" + 0.006*"coronavirus" + 0.005*"biden"
4 - Score: 0.07689768075942993	 Topic: 0.007*"good" + 0.005*"vaccine" + 0.005*"coronavirus" + 0.005*"fuck" + 0.004*"year"


In [ ]:
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3353331387042999	 Topic: 0.043*"mask" + 0.025*"vaccinated" + 0.017*"school" + 0.014*"face" + 0.013*"coronavirus"
Score: 0.2856392562389374	 Topic: 0.054*"case" + 0.042*"coronavirus" + 0.022*"death" + 0.021*"state" + 0.016*"news"
Score: 0.15011870861053467	 Topic: 0.023*"people" + 0.018*"like" + 0.016*"know" + 0.013*"would" + 0.011*"virus"
Score: 0.09828728437423706	 Topic: 0.022*"pandemic" + 0.011*"business" + 0.010*"health" + 0.010*"vaccination" + 0.009*"coronavirus"
Score: 0.05786719173192978	 Topic: 0.044*"death" + 0.038*"test" + 0.035*"case" + 0.029*"positive" + 0.028*"coronavirus"
Score: 0.05675055831670761	 Topic: 0.022*"coronavirus" + 0.020*"year" + 0.009*"pandemic" + 0.008*"china" + 0.008*"covid"


### Visualisation des topics

In [ ]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.085353  0.116661       1        1  25.790992
0      0.001017 -0.168669       2        1  24.813276
2      0.071251  0.065405       3        1  20.067046
3     -0.254459  0.037429       4        1  17.689803
4      0.096837 -0.050827       5        1  11.638882, topic_info=          Term          Freq         Total Category  logprob  loglift
103       case  18158.000000  18158.000000  Default  30.0000  30.0000
182      death  13630.000000  13630.000000  Default  29.0000  29.0000
783      total   4559.000000   4559.000000  Default  28.0000  28.0000
4107   variant   4933.000000   4933.000000  Default  27.0000  27.0000
184   reported   4000.000000   4000.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
179      virus    727.833873   6541.296061   Topic5  -6.3068  -0.0450
219       well    661.870597   5309.839410   Topic5  -6.4018   0.0686
63        know    656.039829   6641.642266   Topic5  -6.4107  -0.1641
36        even    627.586855   4605.806800   Topic5  -6.4550   0.1576
654       read    619.446842   4506.200362   Topic5  -6.4681   0.1664

[452 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2236      1  0.839348  absolutely
2236      2  0.015018  absolutely
2236      3  0.091778  absolutely
2236      4  0.006675  absolutely
2236      5  0.046723  absolutely
...     ...       ...         ...
1160      1  0.002333       yikes
1160      2  0.002333       yikes
1160      3  0.002333       yikes
1160      4  0.993761       yikes
1160      5  0.002333       yikes

[1920 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 4, 5])

In [ ]:
get_document_topics = loaded_model.get_document_topics(corpus_tfidf)
# get_document_topics = lda_model_tfidf.get_document_topics(corpus_tfidf)

### Sauvegarde des données et modèles

In [ ]:
filename = '/content/drive/My Drive/models/lda_model_tfidf.sav'
pickle.dump(lda_model_tfidf, open(filename, 'wb'))

In [ ]:
filename = '/content/drive/My Drive/models/lda_model.sav'
pickle.dump(lda_model, open(filename, 'wb'))

In [ ]:
np.save("/content/drive/My Drive/models/lda_bow.npy", bow_corpus)

In [ ]:
filename = '/content/drive/My Drive/models/lda_model_tfidf.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
bow_corpus = np.load('/content/drive/My Drive/models/lda_bow.npy', allow_pickle=True)

In [ ]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(loaded_model, bow_corpus, dictionary=loaded_model.id2word)
pyLDAvis.save_html(vis,"lda_topic_vis.html")


In [ ]:
def get_tweet_by_df_id(id,df):
  return df.iloc[id,:]['id']
  
cpt=0
docs = dict(list())
topic=4

for i in get_document_topics:
  tmp = sorted(i, key=lambda tup: tup[1])
  if(docs.get(tmp[0][0],None) is None):
     docs[tmp[0][0]]=[]
  docs[tmp[0][0]].append(get_tweet_by_df_id(cpt,df))
  cpt+=1

In [ ]:
filename = '/content/drive/My Drive/models/tweetsIdsByTopic.npy'
pickle.dump(docs, open(filename, 'wb'))

In [ ]:
filename = '/content/drive/My Drive/models/tweetsIdsByTopic.npy'
doc = pickle.load(open(filename, 'rb'))